In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('max_rows', 20)
pd.set_option('max_columns', 40)

import networkx as nx,community
import matplotlib.pyplot as plt
plt.style.use('ggplot')
font = {'family' : 'sans',
        'weight' : 'bold',
        'size'   : 20}
plt.rc('font', **font)

plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams[u'figure.figsize'] = (16,8)

In [25]:
#mas librerias que voy obteniendo
import biopandas.pdb as bp
biop = bp.PandasPdb() #libreria de lectura de pdbs
#libreria de calculo de distancia euclidiana
from scipy.spatial.distance import pdist, squareform
#libreria de mate
import numpy as np

In [116]:
#archivos pdbs
path1 ='/Users/serch/pdbmani/trj_0.pdb'
path2='/Users/serch/pdbmani/trj_0_A.pdb'
# generar mismos resultados que el algoritmo click
pathprueba ='/Users/serch/pdbmani/Serch/1phr.pdb'

#funcion de lectura con biopandas
def read_biopdb(path):
        df = biop.read_pdb(path)
        df_atom = df.df['ATOM']
        #OJO AQUI ESTA ADECUADO AL PDB
        df_ca = df_atom[:1805][df_atom.atom_name == 'CA'][[
        'atom_number','atom_name','residue_name','residue_number',
        'x_coord','y_coord','z_coord']]
        columna_vector = []
        for i in zip(df_ca.x_coord.tolist(),df_ca.y_coord.tolist(),df_ca.z_coord.tolist()):
            columna_vector.append(list(i))
            
        df_ca['vector'] = columna_vector
        return(df_ca)

In [160]:
df_ca1 = read_biopdb(path1)
df_ca2 = read_biopdb(path2)

C:\Users\serch\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


In [161]:
#se calcula la distancia entre cada par de nodos.
# def distancia_entre_atomos(df_ca):
distancias = []
#se calcula la distancia euclidiana entre cada atomo de carbon alfalfa
for v,i in zip(df_ca1.vector,df_ca1.atom_number):
    distancia_un_atomo = []
    for av,j in zip(df_ca1.vector,df_ca1.atom_number):
        distancia = pdist([v,av],metric='euclidean').item()
        distancia_un_atomo.append(distancia)
    distancias.append(distancia_un_atomo)

In [162]:
#se genera la matriz de adyacencias para la red
distancias_adyacenctes = pd.DataFrame(index=df_ca1.atom_number,
                                      columns=df_ca1.atom_number,
                                      data=distancias)

In [163]:
#red de distancias completa
red = nx.from_pandas_adjacency(distancias_adyacenctes)
print(nx.info(red))

Name: 
Type: Graph
Number of nodes: 116
Number of edges: 6670
Average degree: 115.0000


In [164]:
#filtro de distancias
edgesstrong = [(u,v) for (u,v,d) in red.edges(data=True) if d["weight"] <=10]

In [165]:
red = nx.Graph(edgesstrong)

In [166]:
print(nx.info(red))

Name: 
Type: Graph
Number of nodes: 116
Number of edges: 888
Average degree:  15.3103


# Error 1
este codigo siguiente era para generar cliques si coincidian en distancia pero no es adecuado dado que primero hay que generar los cliques y posterior filtrar por distancias.

In [302]:
#el umbral donde formamos los cliques es tomando encuenta dos factores
#1. que el numero de miembros en el clique sea entre 3 y 9, en el articulo lo fijaron en 7 
#2. que la distancia este en el intervalo 6 a 12, en el articulo lo fijaron en 10

def gen_cliques_internos(df_ca):
    #intervalos de distancia posible para formar el clique
    #d_inf,d_sup = 3,9
    # donde se guardaran los posibles cliques
    clique_grande = []
    #se calcula la distancia euclidiana entre cada atomo de carbon alfalfa
    for v,i in zip(df_ca.vector,df_ca.atom_number):
        #clique del atomo a obervarsh
        clique = []
        for av,j in zip(df_ca.vector,df_ca.atom_number):
            if v == av: #si es el mismo atomo no calcular
                continue
            distancia = pdist([v,av],metric='euclidean').item()
            if distancia <= 10: #(d_inf <= distancia) and (distancia <= d_sup) :
    #             print(pdist([v,av],metric='euclidean'),i,j)
                clique = clique + [j]
                #print(clique)
        clique_grande.append(clique)

    df_ca['cliques'] = clique_grande
    return(df_ca)

In [303]:
df_cat1 = gen_cliques_internos(df_ca1)

In [304]:
#se busca saber si un valor esta en otro conjunto y formar los cliques
for i in range(df_cat1.shape[0]):
    for j in range(df_cat1.shape[0]):
        if i == j: continue #no repetidos
        if df_cat1.cliques.values[i] in df_cat1.cliques.values[j]:
            print(set(df_cat1.cliques.values[i]))

In [305]:
df_cat1.set_index('atom_number',inplace=True)

In [306]:
#se genero una red de adyacencias 
#para saber quien conectaba con quien respetando la distancia desde el filtro pasado
lista_enlaces = []
for i in range(df_cat1.shape[0]):
    lista_enlaces = lista_enlaces + [list(df_cat1.index.isin(df_cat1.cliques.values.take(i))*1)]

In [307]:
#se grafico en gephi para ver si estaba bien hecha.
red_adyacencias = pd.DataFrame(index=df_cat1.index,columns=df_cat1.index,data=lista_enlaces)
red_adyacencias.to_csv('red_adyacencias.csv')